In [2]:
import autogen

config_list = autogen.config_list_from_dotenv()

In [3]:
planner = autogen.AssistantAgent(
    name = "planner",
    llm_config={"config_list": config_list},
    system_message="You are a helpful AI assistant. You suggest coding and reasoning steps for another AI assistant to accomplish a task. Do not suggest concrete code. For any action beyond writing code or reasoning, convert it to a step which can be implemented by writing code. For example, the action of browsing the web can be implemented by writing code which reads and prints the content of a web page. Finally, inspect the execution result. If the plan is not good, suggest a better plan. If the execution is wrong, analyze the error and suggest a fix."
)

planner_user = autogen.UserProxyAgent(
    name = "planner_user",
    max_consecutive_auto_reply=0,
    human_input_mode="NEVER",
)

def ask_planner(message):
    planner_user.initiate_chat(planner, message=message)
    
    return planner_user.last_message()["content"]

In [5]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "temperature": 0,
        "request_timeout": 600,
        "seed": 42,
        "model": "gpt-4-0613",
        "config_list": autogen.config_list_openai_aoai(exclude="aoai"),
        "functions": [
            {
                "name": "ask_planner",
                "description": "ask planner to: 1. get a plan for finishing a task, 2. verify the execution result of the plan and potentially suggest new plan.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "question to ask planner. Make sure the question include enough context, such as the code and the execution result. The planner does not know the conversation between you and the user, unless you share the conversation with the planner.",
                        },
                    },
                    "required": ["message"],
                },
            },
        ],
    }
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "planning"},
    function_map={"ask_planner": ask_planner},
)

In [6]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""In Python, provide code that would let me query a vector database. Recommend a vector database to use. Provide an example of adding documents to the vector database.""",
)

user_proxy (to assistant):

In Python, [rovide code that would let me query a vector database. Recommend a vector database to use. Provide an example of adding documents to the vector database.

--------------------------------------------------------------------------------
assistant (to user_proxy):

There are several vector databases available, but one of the most popular and easy to use is Annoy (Approximate Nearest Neighbors Oh Yeah) by Spotify. It's a Python library for searching nearest neighbors in high dimensional spaces. 

First, you need to install the Annoy library. You can do this by running the following command in your terminal:

```sh
pip install annoy
```

After installing Annoy, you can use the following Python code to create an Annoy index, add vectors (documents) to it, and query the index.

```python
from annoy import AnnoyIndex
import random

# Length of the vector that will be indexed
f = 40

# Initialize Annoy with the length of the vector
t = AnnoyIndex(f, 'ang

InvalidRequestError: This model's maximum context length is 8192 tokens. However, your messages resulted in 8458 tokens (8352 in the messages, 106 in the functions). Please reduce the length of the messages or functions.

# New Langauge Model Testing